In [ ]:
import mimicfouretl.bigquery_utils as bq

In [ ]:
bq.set_credentials_file('../bq_credentials/client_secret.json')
bq.set_project_id('mimic-iv-418015')
#bq.set_project_id('micro-vine-412020')

In [ ]:
client = bq.get_client()
# tables = bq.list_tables(client, 'micro-vine-412020.mimic_iv')
tables = bq.list_tables(client, 'physionet-data.mimiciv_hosp')
tables

In [ ]:
spark = bq.get_spark_session(client)

In [ ]:
# results = bq.run_query(spark, "SELECT * FROM `physionet-data.mimiciv_hosp.admissions` LIMIT 10")
results = bq.run_query(spark, "SELECT * FROM `micro-vine-412020.mimic_iv.hosp_admissions` LIMIT 10")

In [ ]:
# bq.display_sampled_df(results)